In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd '/content/drive/MyDrive/Colab Notebooks/CAERS'

/content/drive/MyDrive/Colab Notebooks/CAERS


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Lambda
from tensorflow.keras.models import Model
from keras import losses

from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.metrics import RootMeanSquaredError

In [ ]:
import pandas as pd
import numpy as np
import string
import os
import re
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
movies = pd.read_pickle("/content/drive/MyDrive/Colab Notebooks/Data/movie.csv")
rating = pd.read_pickle("/content/drive/MyDrive/Colab Notebooks/Data/rating.csv")
users  = pd.read_pickle("/content/drive/MyDrive/Colab Notebooks/Data/user.csv")

In [ ]:
movies.reset_index(inplace=True)
movies.rename(columns={'index':'movie_id'},inplace=True)
movies.head(2)

,movie_id,movie_name,genre,wikipedia_page_name,movie_plot
0,1,Toy Story (1995),"[Animation, Children's, Comedy]",Toy Story,A group of living toys are preparing to move i...
1,2,Jumanji (1995),"[Adventure, Children's, Fantasy]",Jumanji,"In 1969, Alan Parrish lives with his parents, ..."


In [ ]:
allData = rating.merge(movies, on='movie_id', how='inner')
allData = allData.merge(users, on='user_id', how='inner')

In [ ]:
allData.dropna(inplace=True)
allData.shape

(935815, 12)

In [ ]:
allData = allData[allData['movie_plot'].apply(len) > 0]
allData.shape

(929231, 12)

In [ ]:
allData['movie_id'].duplicated().sum()


926130

In [ ]:
allData['movie_id'] = pd.factorize(allData['movie_id'])[0] + 1
allData['movie_id'].duplicated().sum()

<ipython-input-50-c4dc9235b4b9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  allData['movie_id'] = pd.factorize(allData['movie_id'])[0] + 1


926130

In [ ]:
movies = allData[['movie_plot', 'genre', 'movie_id','movie_name']]

# Remove duplicates in the 'movies' DataFrame
movies.drop_duplicates(subset=['movie_plot', 'movie_id'], inplace=True)


<ipython-input-51-04feedbc6d20>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies.drop_duplicates(subset=['movie_plot', 'movie_id'], inplace=True)


In [ ]:
allData.to_pickle('/content/drive/MyDrive/Colab Notebooks/Data/allData_1.pkl')

In [ ]:
allData.shape

(929231, 12)

# Movie Embeding

##Stemming Lematizing

In [168]:
# remove punctuation
movies['movie_plot'] = movies['movie_plot'].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))

<ipython-input-168-3f46fdef91b3>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies['movie_plot'] = movies['movie_plot'].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))


In [169]:
df_movies = pd.DataFrame()
movieEmbeding = pd.DataFrame()
df_movies['movie_plot']  = movies['movie_plot'].apply(lambda x: word_tokenize(x.lower()))
df_movies['movie_name'] = movies['movie_name'].apply(lambda x: word_tokenize(x.lower()))
#Stop words
stop_words = set(stopwords.words('english'))
df_movies['movie_plot'] = df_movies['movie_plot'].apply(lambda sentence: [word for word in sentence if word not in stop_words])

#Lemmatizer
lemmatizer = WordNetLemmatizer()
df_movies['movie_plot'] = df_movies['movie_plot'].apply(lambda sentence: [lemmatizer.lemmatize(word) for word in sentence])

#stemming
stemmer = PorterStemmer()
df_movies['movie_plot'] = df_movies['movie_plot'].apply(lambda sentence: [stemmer.stem(word) for word in sentence])


genre_model = Word2Vec(sentences=movies['genre'], vector_size=150, window=5, min_count=1, sg=0)
movieplot_model = Word2Vec(sentences=df_movies['movie_plot'], vector_size=150, window=5, min_count=1, sg=0)
movie_name_model = Word2Vec(sentences=df_movies['movie_name'], vector_size=150, window=5, min_count=1, sg=0)
movieEmbeding['genre'] = movies['genre'].apply(lambda x: sum(genre_model.wv[word] for word in x) / len(x) if len(x) > 0 else [])
movieEmbeding['plot'] = df_movies['movie_plot'].apply(lambda x: sum(movieplot_model.wv[word] for word in x) / len(x) if len(x) > 0 else [])
movieEmbeding['name'] = df_movies['movie_name'].apply(lambda x: sum(movie_name_model.wv[word] for word in x) / len(x) if len(x) > 0 else [])

In [ ]:
movieEmbeding.head()

,genre,plot,name
0,"[-0.0013995122, -0.0019394505, 0.0033830164, 0...","[-0.023771644, -0.24022785, -0.14042655, -0.01...","[0.066883765, -0.064977475, -0.03202548, -0.02..."
1,"[-0.0022986748, -0.007900165, -0.0013379814, -...","[0.006505259, -0.2992991, -0.16796084, -0.0052...","[0.08235628, -0.08250633, -0.039451662, -0.027..."
2,"[0.0049209166, -0.006579022, -0.0024316232, 0....","[-0.021953776, -0.28171283, -0.073863216, -0.0...","[0.079219885, -0.07438459, -0.03408175, -0.023..."
4,"[-0.0028716077, -0.008093305, 3.374713e-05, -0...","[0.01964362, -0.2512326, -0.16082464, -0.02013...","[0.099526875, -0.09865875, -0.048218474, -0.03..."
6,"[-0.0047036903, -0.0024089587, -0.0005393051, ...","[-0.022480497, -0.23026665, -0.12837574, -0.04...","[0.089743376, -0.0890172, -0.042855456, -0.028..."


In [ ]:

movieEmbeding.isna().sum()

genre    0
plot     0
name     0
dtype: int64

In [ ]:
movieEmbeding.shape

(3101, 3)

#User Embeding

In [161]:
users.shape

(6040, 5)

In [162]:
users.tail()

,user_id,gender,age,occupation,zip_code
6035,6036,F,25,scientist,32603
6036,6037,F,45,academic/educator,76006
6037,6038,F,56,academic/educator,14706
6038,6039,F,45,other,01060
6039,6040,M,25,doctor/health care,11106


In [163]:
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
import pandas as pd
from sklearn.preprocessing import MinMaxScaler


userEmbeddings = pd.DataFrame(index=users['user_id'])

# Preprocess and tokenize the "occupation" column
# userEmbeddings['occupation'] = users['occupation'].apply(lambda x: word_tokenize(x.lower()))

# Concatenate the two lists within each row, handling NaN values
# userEmbeddings['combined_occupation'] = userEmbeddings['occupation'].apply(lambda x: x[0] + x[1] if len(x) == 2 else x[0] if len(x) == 1 else [])
# Train Word2Vec model for occupation
model_occ = Word2Vec(sentences=users['occupation'], vector_size=150, window=5, min_count=1, sg=0)

# Train Word2Vec model for gender
model_gen = Word2Vec(sentences=users['gender'].apply(lambda x: [x]), vector_size=150, window=5, min_count=1, sg=0)

# Create userEmbeddings DataFrame

# Use Word2Vec embeddings for occupation and gender
userEmbeddings['occupation'] = users['occupation'].apply(lambda x: sum(model_occ.wv[word] for word in x) / len(x) if len(x) > 0 else [])
userEmbeddings['gender'] = users['gender'].apply(lambda x: model_gen.wv[x])




In [164]:
tokenized_zip_codes = [zip_code.split() for zip_code in users['zip_code']]
model_zip = Word2Vec(sentences=tokenized_zip_codes, vector_size=150, window=5, min_count=1, sg=0)
userEmbeddings['zip_code'] = [model_zip.wv[i] for i in tokenized_zip_codes]

In [165]:
userEmbeddings['zip_code'] = userEmbeddings['zip_code'].apply(lambda x: x[0])

In [170]:
userEmbeddings.tail()

,occupation,gender,zip_code
user_id,,,
6036,"[-0.1684078, -0.24301554, 0.06821139, 0.062303...","[0.005211714, -0.0063401246, -0.00013702075, 0...","[-0.005375391, 0.004782354, -0.0036757851, -0...."
6037,"[-0.1684078, -0.24301554, 0.06821139, 0.062303...","[0.005211714, -0.0063401246, -0.00013702075, 0...","[0.005134836, 0.0058286143, -0.00036919673, -0..."
6038,"[-0.14741698, -0.15093198, 0.11128577, 0.11906...","[0.005211714, -0.0063401246, -0.00013702075, 0...","[0.0049186666, 0.0017307139, -0.00036979915, -..."
6039,"[-0.14989427, -0.19187732, 0.09799826, 0.06758...","[-0.00035748482, 0.0001576209, 0.0034022331, 0...","[-0.0036195524, -0.0049982937, -0.003899916, -..."
6040,NaN,NaN,"[0.0010469103, -0.00651481, -0.0064742845, -0...."


In [171]:
userEmbeddings.isna().sum()

occupation    1
gender        1
zip_code      0
dtype: int64

In [172]:
movieEmbedingNp =  movieEmbeding.to_numpy()
userEmbeddings.dropna(inplace=True)
userEmbeddingNp =  userEmbeddings.to_numpy()


In [173]:
movieEmbeding.head()

,genre,plot,name
0,"[-0.0013995122, -0.0019394505, 0.0033830164, 0...","[-0.031762425, -0.24135487, -0.15285006, 0.009...","[0.058498282, -0.060121465, -0.002848263, 0.00..."
1,"[-0.0022986748, -0.007900165, -0.0013379814, -...","[0.008128872, -0.29128024, -0.16578414, 0.0061...","[0.06984333, -0.07420098, -0.002992428, 0.0068..."
2,"[0.0049209166, -0.006579022, -0.0024316232, 0....","[-0.028659547, -0.29673076, -0.083635785, -0.0...","[0.06822438, -0.0675608, -0.0007875746, 0.0089..."
4,"[-0.0028716077, -0.008093305, 3.374713e-05, -0...","[0.0074587096, -0.24936873, -0.17393366, -0.00...","[0.09164683, -0.09628019, -0.0051071006, 0.011..."
6,"[-0.0047036903, -0.0024089587, -0.0005393051, ...","[-0.030018877, -0.22677666, -0.120199025, -0.0...","[0.07578054, -0.07989303, -0.0036574295, 0.009..."


In [174]:
U = np.array([np.concatenate(col) for col in userEmbeddingNp])
M = np.array([np.concatenate(col) for col in movieEmbedingNp])


In [175]:
U.shape

(6039, 450)

In [176]:
M.shape

(3101, 450)

In [177]:

dot = np.dot(U, M.T)

#Label

In [178]:
# label_data = allData[(allData['user_id']<=2000) & (allData['movie_id']<=2000) ]

In [179]:
labels = allData.pivot(index='user_id', columns='movie_id', values='rating')

In [180]:
sum(labels.isna())

4809651

In [181]:
labels = labels.fillna(0)
labels = labels.drop(labels.index[-1])

In [183]:
labels.shape

(6039, 3101)

In [ ]:
pd.DataFrame(dot).to_pickle('DOT.pkl')
pd.DataFrame(labels).to_pickle('LABELS.pkl')

In [ ]:
dot = pd.read_pickle('DOT.pkl')
labels = pd.read_pickle('LABELS.pkl')

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(dot, labels, test_size=0.2, random_state=122)


In [ ]:
def custom_loss(y_true, y_pred):
    y_true = tf.cast(y_true, dtype=tf.float32)
    y_pred = tf.cast(y_pred, dtype=tf.float32)

    mask = y_true > 0  # Create a mask for non-zero values

    y_true_masked = tf.boolean_mask(y_true, mask)
    y_pred_masked = tf.boolean_mask(y_pred, mask)

    squared_errors = tf.square(y_true_masked - y_pred_masked)
    mse = tf.reduce_mean(squared_errors)

    return mse


In [ ]:
type(X_train)

pandas.core.frame.DataFrame

In [ ]:
X_train = X_train.values[:4800,:3000]
y_train =y_train.values[:4800,:3000]

In [ ]:
X_train = X_train.reshape((1, 4800, 3000, 1))
y_train = y_train.reshape((1, 4800, 3000, 1))


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras.models import Model

def convolutional_autoencoder(input_shape):
    # Encoder
    input_img = Input(shape=input_shape)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(input_img)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    encoded = MaxPooling2D((2, 2), padding='same')(x)

    # Decoder
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(encoded)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)
    decoded = decoded*5

    # Create the autoencoder model
    autoencoder = Model(input_img, decoded)

    return autoencoder

# Assuming your data shape is (4800, 3000, 1)
input_shape = (4800, 3000, 1)

# Create the convolutional autoencoder model
autoencoder = convolutional_autoencoder(input_shape)

# Print the model summary
autoencoder.summary()


Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 4800, 3000, 1)]   0         
                                                                 
 conv2d_5 (Conv2D)           (None, 4800, 3000, 64)    640       
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 2400, 1500, 64)    0         
 g2D)                                                            
                                                                 
 conv2d_6 (Conv2D)           (None, 2400, 1500, 32)    18464     
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 1200, 750, 32)     0         
 g2D)                                                            
                                                                 
 conv2d_7 (Conv2D)           (None, 1200, 750, 32)     9248

In [ ]:
import numpy as np

# Assuming you have your data, X_train, with shape (num_samples, 3094, 1, 1)

# Set the batch size
# Compile the model
autoencoder.compile(optimizer='adam', loss=custom_loss)

# Fit the model with your data
autoencoder.fit(X_train, y_train, epochs=20, batch_size=32, shuffle=True)


Epoch 1/20
1/1 [==============================] - 2s 2s/step - loss: 2.3942
Epoch 2/20
1/1 [==============================] - 0s 183ms/step - loss: 2.3106
Epoch 3/20
1/1 [==============================] - 0s 177ms/step - loss: 2.2036
Epoch 4/20
1/1 [==============================] - 0s 177ms/step - loss: 2.0609
Epoch 5/20
1/1 [==============================] - 0s 186ms/step - loss: 1.8800
Epoch 6/20
1/1 [==============================] - 0s 177ms/step - loss: 1.6631
Epoch 7/20
1/1 [==============================] - 0s 177ms/step - loss: 1.4385
Epoch 8/20
1/1 [==============================] - 0s 177ms/step - loss: 1.2740
Epoch 9/20
1/1 [==============================] - 0s 177ms/step - loss: 1.2754
Epoch 10/20
1/1 [==============================] - 0s 179ms/step - loss: 1.4155
Epoch 11/20
1/1 [==============================] - 0s 177ms/step - loss: 1.4803
Epoch 12/20
1/1 [==============================] - 0s 177ms/step - loss: 1.4421
Epoch 13/20
1/1 [==============================] - 0

In [ ]:
dot_ = dot.values[:6000,:3100]
labels_ = labels.values[:6000,:3100]

In [ ]:
dot_ = dot_.reshape((1, 6000, 3100, 1))
labels_ = labels_.reshape((1, 6000, 3100, 1))

In [ ]:
predictions = autoencoder.predict(dot_)

1/1 [==============================] - 0s 53ms/step


In [ ]:
predictions.shape

(1, 6000, 3100, 1)

In [ ]:
mask = labels_ > 0  # Create a mask for non-zero values

In [ ]:
predictions = predictions * mask

In [ ]:
predictions= predictions.reshape((6000, 3100))

In [ ]:
pred = pd.DataFrame(predictions, index=labels.index[:6000],columns=labels.columns[:3100])

In [ ]:
pred.to_pickle('predictions_2.pkl')

In [ ]:
# import tensorflow as tf
# from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, UpSampling1D
# from tensorflow.keras.models import Model

# def convolutional_autoencoder_1d(input_shape):
#     # Encoder
#     input_seq = Input(shape=input_shape)
#     x = Conv1D(64, 3, activation='relu', padding='same')(input_seq)
#     x = MaxPooling1D(2, padding='same')(x)
#     x = Conv1D(32, 3, activation='relu', padding='same')(x)
#     encoded = MaxPooling1D(2, padding='same')(x)

#     # Decoder
#     x = Conv1D(32, 3, activation='relu', padding='same')(encoded)
#     x = UpSampling1D(2)(x)
#     x = Conv1D(64, 3, activation='relu', padding='same')(x)
#     x = UpSampling1D(2)(x)
#     decoded = Conv1D(1, 3, activation='sigmoid', padding='same')(x)
#     decoded =decoded*5

#     # Create the autoencoder model
#     autoencoder = Model(input_seq, decoded)

#     return autoencoder

# # Assuming your data shape is (4800, 3000)
# input_shape_1d = (3000, 1)

# # Create the 1D convolutional autoencoder model
# autoencoder_1d = convolutional_autoencoder_1d(input_shape_1d)

# # Print the model summary
# autoencoder_1d.summary()

# # Compile the model
# autoencoder_1d.compile(optimizer='adam', loss=custom_loss)

# # Reshape your data accordingly
# X_train_1d = X_train.reshape((4800, 3000, 1))
# y_train_1d = y_train.reshape((4800, 3000, 1))

# # Fit the model with your data
# autoencoder_1d.fit(X_train_1d, y_train_1d, epochs=20, batch_size=32, shuffle=True)
